In [1]:
# حذف بسته‌های مشکل‌ساز
%pip uninstall -y numpy pandas torch transformers datasets hazm tokenizers accelerate pyarrow

# نصب پکیج‌های اصلی
%pip install numpy==1.26.2 pandas==2.2.2
%pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install datasets==2.14.0 tokenizers==0.15.2
%pip install transformers==4.41.0
%pip install pyarrow==14.0.2
%pip install git+https://github.com/sobhe/hazm.git
%pip install accelerate
%pip install ipywidgets

Found existing installation: numpy 1.26.2
Uninstalling numpy-1.26.2:
  Successfully uninstalled numpy-1.26.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: torch 2.7.1+cpu
Uninstalling torch-2.7.1+cpu:
  Successfully uninstalled torch-2.7.1+cpu
Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0
Found existing installation: datasets 2.14.0
Uninstalling datasets-2.14.0:
  Successfully uninstalled datasets-2.14.0
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Note: you may need to restart the kernel to use updated 

  Using cached numpy-1.26.2-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached numpy-1.26.2-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.0 requires pyarrow>=7.0.0, which is not installed.
streamlit 1.32.0 requires pyarrow>=7.0, which is not installed.
torchvision 0.22.1+cpu requires torch==2.7.1, which is not installed.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.


Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/216.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/216.0 MB 2.5 MB/s eta 0:01:28
   ---------------------------------------- 0.3/216.0 MB 2.9 MB/s eta 0:01:15
   ---------------------------------------- 0.5/216.0 MB 3.0 MB/s eta 0:01:12
   ---------------------------------------- 0.6/216.0 MB 3.0 MB/s eta 0:01:13
   ---------------------------------------- 0.7/216.0 MB 2.7 MB/s eta 0:01:19
   ---------------------------------------- 0.8/216.0 MB 2.7 MB/s eta 0:01:20
   ---------------------------------------- 1.0/216.0 MB 2.8 MB/s eta 0:01:18
   ---------------------------------------- 1.1/216.0 MB 2.9 MB/s eta 0:01:16
   ---------------------------------------- 1.2/216.0 MB 2.9 MB/s eta 0:01:15
   ---------------------------------------- 1.2/216.0 MB 2.9 MB/s eta 0:01:15
   ------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.


  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-14.0.2-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
Using cached pyarrow-14.0.2-cp312-cp312-win_amd64.whl (25.0 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 21.0.0
    Uninstalling pyarrow-21.0.0:
      Successfully uninstalled pyarrow-21.0.0
Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-14.0.2-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
Using cached pyarrow-14.0.2-cp312-cp312-win_amd64.whl (25

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.


  Cloning https://github.com/sobhe/hazm.git to c:\users\alireza\appdata\local\temp\pip-req-build-vg_ij8dl
  Resolved https://github.com/sobhe/hazm.git to commit d97a5ba13a680a29f2ae5bc6f862e941329890fb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fasttext_wheel-0.9.2-cp312-cp312-win_amd64.whl.metadata (16 kB)
  Using cached flashtext-2.7.tar.gz (14 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished w

  Running command git clone --filter=blob:none --quiet https://github.com/sobhe/hazm.git 'C:\Users\Alireza\AppData\Local\Temp\pip-req-build-vg_ij8dl'
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Alireza\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\Alireza\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Alireza\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        F

  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.9.0-py3-none-any.whl (367 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
    import numpy as np
    import pandas as pd
    import torch
    from hazm import Normalizer
    from datasets import Dataset
    import transformers
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

    print("✅ کتابخانه‌ها با موفقیت import شدند!")
    print(f"🔹 numpy: {np.__version__}")
    print(f"🔹 pandas: {pd.__version__}")
    print(f"🔹 torch: {torch.__version__}")
    print(f"🔹 transformers: {transformers.__version__}")

except Exception as e:
    print("❌ خطا در import:", e)

❌ خطا در import: No module named 'hazm'


In [ ]:
import pandas as pd

# لینک‌ها
url1 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part1.csv"
url2 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part2.csv"

# 📥 خواندن فایل‌ها با تنظیمات دقیق
df1 = pd.read_csv(url1, sep="\t", header=0)
df2 = pd.read_csv(url2, sep="\t", header=0)

# 🔗 ادغام
df = pd.concat([df1, df2], ignore_index=True)

# حذف ستون‌های اضافه‌ای که اشتباه وارد شدن
expected_columns = ['comment_normalized', 'hate', 'spam', 'obscene', 'class']
df = df[expected_columns]

# نمایش اولیه
print(f"🔢 تعداد کل داده‌ها: {len(df)}")
print("🧾 ستون‌ها:", df.columns.tolist())
print(df.head(3))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# حذف ردیف‌هایی که مقدار کلاس ندارن
df = df.dropna(subset=["class"])

# دوباره لیبل‌ها رو بساز
label_encoder = LabelEncoder()
df.loc[:, "label"] = label_encoder.fit_transform(df["class"])

# نمایش کلاس‌ها
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("🔢 کلاس‌ها:", label_map)

# تبدیل به Dataset
dataset = Dataset.from_pandas(df[["comment_normalized", "label"]])
print("✅ Dataset آماده است!")

In [ ]:
from hazm import Normalizer
from transformers import AutoTokenizer

normalizer = Normalizer()
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)س

def preprocess_data(texts):
    return tokenizer([normalizer.normalize(t) for t in texts], truncation=True, padding=True)

print("✅ توکنایزر و نرمالایزر آماده است!")


In [ ]:

encodings = preprocess_data(df['text'].tolist())
labels = df['label'].tolist()

dataset = Dataset.from_dict({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': labels
})

splits = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits['train']
eval_dataset = splits['test']

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./phicad_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    logging_dir='./phicad_logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

model.save_pretrained("phicad_model")
tokenizer.save_pretrained("phicad_model")

In [ ]:
def is_inappropriate(text):
    t = normalizer.normalize(text)
    inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    return torch.argmax(logits, dim=1).item() == 1

comments = [
    "لعنت به این وضعیت خرابات!",
    "خیلی ممنون از خدمات شما"
]
for c in comments:
    print(c, "→", "نامناسب" if is_inappropriate(c) else "مناسب")